In [ ]:
# ✅ 1. 설치 및 인증
!pip install --upgrade openai==0.28 gspread gspread_dataframe oauth2client umap-learn

from google.colab import auth
auth.authenticate_user()

import pandas as pd
import numpy as np
import gspread
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from gspread_dataframe import set_with_dataframe
from google.auth import default
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import MDS
from sklearn.metrics import silhouette_score
from matplotlib.patches import Patch
from collections import defaultdict
import openai
import re

# ✅ 2. 구글 시트 연결
creds, _ = default()
gc = gspread.authorize(creds)
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1kkt336f1G-XqfDuwCUOnqpKlxTcnwLQy-XS4SQv6lM0/edit"
sh = gc.open_by_url(spreadsheet_url)

survey_ws = sh.worksheet("설문지 응답 시트")
score_ws  = sh.worksheet("과목별 직무 가중치1")

# ✅ 3. 데이터 불러오기
survey_df = pd.DataFrame(survey_ws.get_all_records())
score_df  = pd.DataFrame(score_ws.get_all_records()).set_index("과목명")
industries = list(score_df.columns)

# ✅ 4. GPT API 키 설정
openai.api_key = ""

# ✅ 5. GPT 점수 생성 함수
def gpt_industry_score(text):
    prompt = f"""
다음은 공대생의 프로젝트 또는 활동 설명입니다.
아래 산업군 목록 중 관련된 항목이 있다면 반드시 1~3점(정수)으로 점수를 부여하세요.
- 반드시 아래 항목명만 사용하세요.
- 중요도가 낮아도 1점 이상 부여하세요.

[산업군 목록]
{', '.join(industries)}

설명:
{text}

예시:
{industries[0]}: 2, {industries[1]}: 1, {industries[2]}: 3
"""
    try:
        resp = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        content = resp.choices[0].message.content
        print("🤖 GPT 응답:\n", content)

        scores = pd.Series(0, index=industries, dtype=float)
        parts = re.split(r"[,\n]", content)
        for part in parts:
            match = re.search(r"(.+?):\s*([\d.]+)", part)
            if match:
                k = match.group(1).strip()
                v = float(match.group(2))
                if k in scores:
                    scores[k] = min(max(v, 1.0), 3.0)
        return scores
    except Exception as e:
        print("GPT 오류:", e)
        return pd.Series(0, index=industries, dtype=float)

# ✅ 6. 점수 계산 함수
def calculate_combined_scores(row):
    s = pd.Series(0, index=industries, dtype=float)
    subj_score = pd.Series(0, index=industries, dtype=float)
    gpt_score = pd.Series(0, index=industries, dtype=float)

    subjects = [x.strip() for x in row["수강 한 전공 과목 (복수 응답 가능)"].split(',')]
    for subj in subjects:
        if subj in score_df.index:
            subj_score += score_df.loc[subj]
    s += subj_score

    text = row.get("기타활동/프로젝트 경험(주관식)")
    if pd.notna(text) and len(text.strip()) > 5:
        gpt_score = gpt_industry_score(text)
        s += gpt_score

    if s.sum() > 0:
        s = s / s.sum() * 100

    return pd.concat([
        pd.Series({
            "객관식 점수 합계": subj_score.sum(),
            "주관식 점수 합계": gpt_score.sum()
        }),
        subj_score.add_prefix("객관식_"),
        gpt_score.add_prefix("주관식_"),
        s
    ])

# ✅ 7. 산업군 점수 계산
ind_scores = survey_df.apply(calculate_combined_scores, axis=1)
df_scores = pd.concat([survey_df[["이메일 주소", "이름"]], ind_scores], axis=1)

# ✅ 8. 클러스터링
X = df_scores[industries].fillna(0).values
X_scaled = StandardScaler().fit_transform(X)

best_k, best_score = 0, -1
for k in range(2, 7):
    km = KMeans(n_clusters=k, random_state=42)
    labels = km.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    if score > best_score:
        best_k, best_score, best_kmeans, best_labels = k, score, km, labels

kmeans = best_kmeans
clusters = best_labels
df_scores["Cluster"] = clusters
df_scores["Distance_to_Center"] = kmeans.transform(X_scaled)[np.arange(len(X)), clusters]

# ✅ 9. 클러스터 이름 생성 및 연결
def get_cluster_names(centers, top_n=2):
    names = []
    for i in range(len(centers)):
        top = centers.iloc[i].sort_values(ascending=False).head(top_n).index
        names.append(" + ".join(top))
    return names

centers = pd.DataFrame(kmeans.cluster_centers_, columns=industries)
cluster_names = get_cluster_names(centers)
df_scores["Top Industries"] = df_scores["Cluster"].map(lambda c: cluster_names[c])

# ✅ 10. 이상치 탐지
m_dist = df_scores["Distance_to_Center"].mean()
s_dist = df_scores["Distance_to_Center"].std()
outlier_thresh = m_dist + 2 * s_dist
df_scores["Outlier"] = df_scores["Distance_to_Center"] > outlier_thresh

# ✅ 11. MDS 시각화 (1차 군집)
X_aug = np.vstack([X_scaled, kmeans.cluster_centers_])
mds = MDS(n_components=2, random_state=42)
X_mds_all = mds.fit_transform(X_aug)
X_mds = X_mds_all[:-best_k]
centroids_2d = X_mds_all[-best_k:]

df_scores["MDS1"] = X_mds[:, 0]
df_scores["MDS2"] = X_mds[:, 1]

# 색상 매핑
def get_cluster_colors(n_clusters):
    colormap = cm.get_cmap('tab20', n_clusters)
    return [mcolors.rgb2hex(colormap(i)) for i in range(n_clusters)]
colors = get_cluster_colors(best_k)

# 시각화 - 1차 클러스터
plt.figure(figsize=(10, 7))
for i in range(best_k):
    mask = df_scores["Cluster"] == i
    plt.scatter(df_scores.loc[mask, "MDS1"], df_scores.loc[mask, "MDS2"],
                label=cluster_names[i], color=colors[i], s=70)
plt.scatter(centroids_2d[:, 0], centroids_2d[:, 1], marker='X', c='black', s=200, label="Centers")
plt.title("MDS 2D Clustering Result (Primary Clustering)")
plt.legend(fontsize='small')
plt.grid(True)
plt.xlabel("MDS 1")
plt.ylabel("MDS 2")
plt.show()

# ✅ 12. Subgroup 군집화 (6~10명 기준)
df_scores["Subgroup"] = ""
for c in range(best_k):
    members = df_scores[df_scores["Cluster"] == c]
    n = len(members)

    if n > 10:
        n_sub = int(np.round(n / 8))
        n_sub = max(1, min(n_sub, n // 6))
        sub_X = members[["MDS1", "MDS2"]].values
        sub_kmeans = KMeans(n_clusters=n_sub, random_state=42).fit(sub_X)
        for i, idx in enumerate(members.index):
            df_scores.at[idx, "Subgroup"] = f"{cluster_names[c]} Subgroup {sub_kmeans.labels_[i]+1}"
    else:
        for idx in members.index:
            df_scores.at[idx, "Subgroup"] = f"{cluster_names[c]} Subgroup 1"

# ✅ 13. Subgroup 시각화
subgroups = df_scores["Subgroup"].unique()
sub_color_map = {sg: cm.tab20(i / len(subgroups)) for i, sg in enumerate(subgroups)}
df_scores["Sub_Color"] = df_scores["Subgroup"].map(sub_color_map)

plt.figure(figsize=(10, 7))
plt.scatter(df_scores["MDS1"], df_scores["MDS2"], c=df_scores["Sub_Color"], s=70, edgecolors='k')
handles = [Patch(color=sub_color_map[sg], label=sg) for sg in sorted(subgroups)]
plt.legend(handles=handles, title="Subgroups", loc='best', fontsize='small')
plt.title("MDS 2D Subgroup Clustering")
plt.xlabel("MDS 1")
plt.ylabel("MDS 2")
plt.grid(True)
plt.show()

# ✅ 14. Google Sheet 업로드
try:
    ws_old = sh.worksheet("Clustered Result with Distance")
    sh.del_worksheet(ws_old)
except:
    pass
result_ws = sh.add_worksheet(title="Clustered Result with Distance", rows=len(df_scores)+1, cols=80)
set_with_dataframe(result_ws, df_scores)

print(f"✅ 완료: k={best_k}로 클러스터링 + Subgroup 완료 및 시트 업로드")